<a href="https://colab.research.google.com/github/FuturisticLover123/labs/blob/main/lab8/solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [222]:
# task1
from sklearn.metrics import accuracy_score
import numpy as np

class NBC:
  def __init__(self):
    pass


  def pxy(self, ditribution, x, cld):
    if ditribution == "gaussian":
      m, q = cld
      return (1 / (q * np.sqrt(2 * np.pi)) ) * np.exp(-((x - m)**2 / (2 * q**2)))



  def get_cl_data(self, distibution, x, y):
    if distibution == "gaussian":
      return np.mean(x), np.std(x)

  def fit(self, x, y, feature_distribution: dict = {}):
    self.x, self.y = x, y
    self.classes = list(set(y))
    self.feature_distribution = feature_distribution
    indexes = { clas : np.where(y == clas)[0] for clas in self.classes }

    self.prior_probability = { clas : len(indexes[clas]) / len(y) for clas in indexes }

    self.classifying_data = dict()
    for clas in self.classes:
      self.classifying_data[clas] = self.classifying_data.get(clas, dict())
      for feature in range(x.shape[1]):
        distribution = self.feature_distribution.get(feature, "gaussian")
        self.classifying_data[clas][feature] = self.get_cl_data(distribution, x[indexes[clas]][: ,feature], y[indexes[clas]])
    return self

  def classify(self, x):
    weights = { clas : np.log(self.prior_probability[clas]) for clas in self.classes }
    for clas in self.classes:
      for feature in range(len(x)):
        distribution = self.feature_distribution.get(feature, "gaussian")
        weights[clas] += np.log( self.pxy(distribution, x[feature], self.classifying_data[clas][feature]) )
    return max(weights, key=weights.get)

  def predict(self, x):
    return np.array([self.classify(x_i) for x_i in x])

  def score(self, x, y):
    y_pred = self.predict(x)
    return accuracy_score(y, y_pred)

  def get_params(self, deep=True):
    return {
    }

  def set_params(self, **params):
    for key, value in params.items():
        setattr(self, key, value)
    return self


In [223]:
from sklearn.datasets import make_classification

In [224]:
x,y = make_classification(n_samples = 1000,n_features = 30,n_redundant=20)

In [225]:
NBC().fit(x, y).score(x, y)

0.909

In [226]:
#task2

In [227]:
import pandas as pd

In [79]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [80]:
!kaggle datasets download -d valakhorasani/mobile-device-usage-and-user-behavior-dataset -p /content/sample_data/lab8
!unzip /content/sample_data/lab8/mobile-device-usage-and-user-behavior-dataset.zip -d /content/sample_data/lab8

Dataset URL: https://www.kaggle.com/datasets/valakhorasani/mobile-device-usage-and-user-behavior-dataset
License(s): apache-2.0
  0% 0.00/11.3k [00:00<?, ?B/s]
100% 11.3k/11.3k [00:00<00:00, 20.4MB/s]
Archive:  /content/sample_data/lab8/mobile-device-usage-and-user-behavior-dataset.zip
  inflating: /content/sample_data/lab8/user_behavior_dataset.csv  


In [98]:
data = pd.read_csv("/content/sample_data/lab8/user_behavior_dataset.csv").dropna()

In [99]:
data.head(3)

,User ID,Device Model,Operating System,App Usage Time (min/day),Screen On Time (hours/day),Battery Drain (mAh/day),Number of Apps Installed,Data Usage (MB/day),Age,Gender,User Behavior Class
0,1,Google Pixel 5,Android,393,6.4,1872,67,1122,40,Male,4
1,2,OnePlus 9,Android,268,4.7,1331,42,944,47,Female,3
2,3,Xiaomi Mi 11,Android,154,4.0,761,32,322,42,Male,2


In [100]:
data["Device Model"] = data["Device Model"].apply(lambda x: x.split(' ')[0])

In [105]:
x = pd.get_dummies(data, columns = ['Device Model', 'Operating System', 'Gender']).drop(columns = ['User ID', 'User Behavior Class']).astype(float).values
y = data['User Behavior Class'].values

In [106]:
from sklearn.model_selection import cross_val_score

In [109]:
model = NBC()
cross_val_score(model, x, y, cv=10).mean()

1.0

In [522]:
#task 3

In [523]:
data = pd.read_csv('/content/sample_data/lab8/spam.csv', encoding='ISO-8859-1').drop(columns = ['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']).dropna()

In [460]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [461]:
def preprocess_text(text):
    text = text.lower()

    text = re.sub(r'[^\w\s]', '', text)

    words = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    return ' '.join(words)


text = "I running love programming in Python, and I enjoy solving coding challenges!"

processed_text = preprocess_text(text)
print(processed_text)


run love program python enjoy solv code challeng


In [524]:
data['v2'] = data['v2'].apply(preprocess_text)

In [525]:
x = data['v2'].values
y = data['v1'].map({ 'spam' : 0, 'ham':1 }).values

In [546]:
class SpamNBC(NBC):
  def __init__(self, alpha=1):
    super().__init__()
    self.alpha=alpha

  def fit(self, x, y):
    self.x, self.y = x, y
    self.classes, clf_counts = np.unique(y, return_counts=True)
    self.clf_counts = {c: clf for c, clf in zip(self.classes, clf_counts)}
    self.prior = {c: clf / len(y) for c, clf in zip(self.classes, clf_counts)}

    self.frequency = dict()
    self.words_in_class = dict()

    for c in self.classes:
      self.frequency[c] = dict()
      self.words_in_class[c] = set()

      for message in x[np.where(y == c)[0]]:
        for word in message.split(' '):
          self.words_in_class[c].add(word)
          self.frequency[c][word] = self.frequency[c].get(word, 0) + 1
    self.words_in_class = {c : len(self.words_in_class[c]) for c in self.classes}

    self.ni = len(set(' '.join(x).split(' ')))
    return self

  def classify(self, x):
    weights = { c: np.log(self.prior[c]) for c in self.prior }
    for c in self.classes:
      for word in x:
        weights[c] += np.log((self.frequency[c].get(word, 0) + self.alpha) / (self.words_in_class[c] + self.ni * self.alpha))
    print(weights)
    return max(weights, key=weights.get)

In [550]:
model = SpamNBC(alpha=1).fit(x, y)

In [551]:
model.frequency[1]['win']

18

In [552]:
sum(model.predict(x))

Выходные данные были обрезаны до нескольких последних строк (5000).
{0: -76.81319881374303, 1: -66.44596636102148}
{0: -80.67082774710538, 1: -77.94455643917715}
{0: -181.68059771192011, 1: -175.67321114211305}
{0: -337.95814838342017, 1: -344.6171223109631}
{0: -261.9582482337329, 1: -231.25220370868743}
{0: -224.05883414360267, 1: -219.31749795846292}
{0: -551.7302228014474, 1: -530.8337532898196}
{0: -826.598571339923, 1: -769.3466874461363}
{0: -271.52130132578054, 1: -256.49555925337216}
{0: -218.04185547625917, 1: -200.0962601609029}
{0: -858.6835736171654, 1: -801.2315706883701}
{0: -263.8775755098217, 1: -243.6037882881434}
{0: -464.49718555771176, 1: -425.99319957224327}
{0: -330.34036260155267, 1: -310.26299328803407}
{0: -589.9423278653704, 1: -560.3391317361906}
{0: -343.16130514459917, 1: -323.8477126355294}
{0: -189.49493975638805, 1: -185.54358512405838}
{0: -153.8853472059649, 1: -146.76515231395115}
{0: -767.0497476534541, 1: -720.2888051441777}
{0: -789.2496482667999,

5558

In [553]:
x.shape

(5572,)

In [554]:
model.score(x, y)

Выходные данные были обрезаны до нескольких последних строк (5000).
{0: -76.81319881374303, 1: -66.44596636102148}
{0: -80.67082774710538, 1: -77.94455643917715}
{0: -181.68059771192011, 1: -175.67321114211305}
{0: -337.95814838342017, 1: -344.6171223109631}
{0: -261.9582482337329, 1: -231.25220370868743}
{0: -224.05883414360267, 1: -219.31749795846292}
{0: -551.7302228014474, 1: -530.8337532898196}
{0: -826.598571339923, 1: -769.3466874461363}
{0: -271.52130132578054, 1: -256.49555925337216}
{0: -218.04185547625917, 1: -200.0962601609029}
{0: -858.6835736171654, 1: -801.2315706883701}
{0: -263.8775755098217, 1: -243.6037882881434}
{0: -464.49718555771176, 1: -425.99319957224327}
{0: -330.34036260155267, 1: -310.26299328803407}
{0: -589.9423278653704, 1: -560.3391317361906}
{0: -343.16130514459917, 1: -323.8477126355294}
{0: -189.49493975638805, 1: -185.54358512405838}
{0: -153.8853472059649, 1: -146.76515231395115}
{0: -767.0497476534541, 1: -720.2888051441777}
{0: -789.2496482667999,

0.8645010768126346

In [497]:
from sklearn.naive_bayes import MultinomialNB

In [545]:
np.log(2.7)

0.9932517730102834

In [516]:
x = np.array(["бесплатно получите скидку", "получите деньги", "встретимся завтра"])

y = np.array([0,0,1])

In [517]:
model = SpamNBC(alpha=1).fit(x, y)

{'скидку', 'бесплатно', 'встретимся', 'получите', 'деньги', 'завтра'}


In [518]:
model.prior

{0: 0.6666666666666666, 1: 0.3333333333333333}

In [519]:
model.frequency[0]["получите"]

2

In [520]:
model.words_in_class

{0: 4, 1: 2}

In [502]:
model.words_in_class

{0: 5, 1: 2}

In [493]:
model.ni


6